# 経営情報各論BⅡ：8. 検定

## (3)平均値の差の検定
### ２群のデータに対する$t$検定
今までは，１種類のスナック菓子の重量といった「一変量のデータ」だけを対象としていました．  
次は2つの変数の間で，平均値に差があるかどうかを判断します．

例えば，薬を飲む前と飲んだ後で，体温に差が出るかどうかを調べる場合．あるいは，大きい針で釣った魚と小さい針で釣った魚とで，釣れた魚の体長に差がでるかどうかを調べる場合，などに使用されます．

### 対応のある$t$検定
例えば，薬を飲む前と飲んだ後で，体温に差が出るかどうかを調べる場合など，「同じ対象を，異なった条件で2回測定して，その違いを見る」といった場合に**対応のある$t$検定**を使います．

対応のある$t$検定の例として，以下の架空の調査データを使います．

|被験者|薬を飲む前の体温|薬を飲んだ後の体温|差分|
|:---:|:---:|:---:|:---:|
|Aさん|36.2|36.8|0.6|
|Bさん|36.2|36.1|-0.1|
|Cさん|35.3|36.8|1.5|
|Dさん|36.1|37.1|1.0|
|Eさん|36.1|36.9|0.8|

この時，一番右の列の「薬を飲む前と後の体温の差分」に注目します．もしも，薬が体温に何の影響も与えていないのであれば，この差分の値が0になるはずです．逆に言えば，**差分の列の平均値が0と異なれば「薬を飲む前と飲んだ後で体温が異なる」ということが主張できる**わけです．

対応のある$t$検定では，このように，差分をとってから「差分値が0と有意に異なるか」という1群の$t$検定を行います．

### t検定の実装：分析の準備
まずは，必要なライブラリの読み込みや表示桁数の設定などを行います．

In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# グラフを描画するライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# 表示桁数の指定
%precision 3
# グラフをjupyter Notebook内に表示させるための指定
%matplotlib inline

続いて，今回の分析の対象となるデータを読み込みます．  
薬を飲む前と後の体温を測定した架空のデータです．サンプルサイズは10です．データフレーム形式で読み込みます．

In [2]:
# データの読み込み
paired_test_data = pd.read_csv(
    "ds2-testing-3-paired-t-test.csv")
print(paired_test_data)

  person medicine  body_temperature
0      A   before              36.2
1      B   before              36.2
2      C   before              35.3
3      D   before              36.1
4      E   before              36.1
5      A    after              36.8
6      B    after              36.1
7      C    after              36.8
8      D    after              37.1
9      E    after              36.9


このデータを対象に，対応のある$t$検定を実行します．  
以下の要領で検定を行います．  
帰無仮説：薬を飲む前と後で体温は変わらない  
対立仮説：薬を飲む前と後の体温が異なっている  
有意水準は$5\%$とします．$p$値が0.05を下回れば，帰無仮説は棄却され，薬を飲むことで体温の有意な変化が認められると主張できることになります．

### 実装：対応のあるt検定
薬を飲む前と飲んだ後における，体温の差を計算します．シリーズ型のままだと計算がしにくいので，薬を飲む前・後で抽出した後，アレイ型に変換しました．

In [3]:
# 薬を飲む前と飲んだ後の標本平均
before = paired_test_data.query(
    'medicine == "before"')["body_temperature"]
after = paired_test_data.query(
    'medicine == "after"')["body_temperature"]
# アレイに変換
before = np.array(before)
after = np.array(after)
# 差を計算
diff = after - before
diff

array([ 0.6, -0.1,  1.5,  1. ,  0.8])

後は，この差の値の平均値が0と異なるかどうかを，1群の$t$検定で調べればよいです．

In [4]:
# 平均値が0と異なるか検定
stats.ttest_1samp(diff, 0)

TtestResult(statistic=2.901693483620596, pvalue=0.044043109730074276, df=4)

`stats.ttest_rel`関数を使えば簡単に検定をできます．

In [5]:
# 対応のあるt検定
stats.ttest_rel(after, before)

TtestResult(statistic=2.901693483620596, pvalue=0.044043109730074276, df=4)

$p$値が0.05を下回ったので「薬を飲む前と後の体温は有意に異なる」と主張できることになる．

### 対応の無い$t$検定
次は，対応の無い$t$検定の仕組みを説明します．  
対応の無い$t$検定は「平均値の差」に注目します．  
対応のある$t$検定だと「データの差」をとってから1群の$t$検定をかけていました．この違いがあるということです．  
平均値の差に基づいて$t$値を計算する場合は，$t$値の計算式が若干複雑になります．  
1群の$t$検定における$t$値の計算式を再掲します．

$\displaystyle t値=\frac{標本平均-比較対象値}{標準偏差\div\sqrt{サンプルサイズ}}=\frac{標本平均-比較対象値}{標準誤差}$

変数$x$と$y$の平均値の差を検定するとしましょう．変数$x$は例えば「大きい針で釣った魚の体長」で変数$y$は「小さい針で釣った魚の体長」などとなります．

対応の無い$t$検定の$t$値は以下のように計算されます．

$\displaystyle t=\frac{\hat{\mu_x}-\hat{\mu_y}}{\sqrt{\frac{\hat{\sigma_x}^2}{m}+{\frac{\hat{\sigma_y}^2}{n}}}}$

ただし，$\hat{\mu_x}$は$x$の標本平均で，$\hat{\mu_y}$は$y$の標本平均です．  
$m$は$x$のサンプルサイズで，$n$は$y$のサンプルサイズです．  
また，$\hat{\sigma_x}^2$は$x$の不偏分散で，$\hat{\sigma_y}^2$は$y$の不偏分散です．  
大体は1群の$t$検定における$t$値と似たような感じではあります．ただ，2つの変数で分散が異なっていることを仮定したうえで計算をしているため，分母の標準誤差がやや複雑になっています．

後は，これを使って$t$検定を行います．  
ただし，2群の場合は$t$分布の自由度が複雑になります．Welchの近似法と呼ばれる方法を使って$p$値を計算することになります．このため，この方法をWelchの検定と呼びます．

### 実装：対応の無い$t$検定
検定します．データは「対応ある$t$検定」と同じものを使うことにします．もちろんこれは勉強のためです．本来はデータに合わせて最適な検定手法を選ぶ必要があるので，対応の無い$t$検定は「対応の無いデータ」に適用するべきです．

$t$値を計算します．

In [6]:
# 平均値
mean_bef = np.mean(before)
mean_aft = np.mean(after)

# 分散
sigma_bef = np.var(before, ddof = 1)
sigma_aft = np.var(after, ddof = 1)

# サンプルサイズ
m = len(before)
n = len(after)

# t値
t_value = (mean_aft - mean_bef) / \
    np.lib.scimath.sqrt((sigma_bef/m + sigma_aft/n))
t_value

3.156

対応の無い$t$検定は`stats.ttest_ind`関数を使えば簡単に計算できます．

In [7]:
stats.ttest_ind(after, before, equal_var = False)

Ttest_indResult(statistic=3.1557282344421034, pvalue=0.013484775682079892)

$p$値が0.05を下回ったので，やはり有意差があると判断できる結果となりました．  
しかし，$p$値が「対応のある$t$検定」の結果（0.04ほど）と異なっています．あたりまえといえばあたりまえですが，同じデータに対して同じ目的の検定を行っても，検定の手法が変わると$p$値も変わるということです．

### 対応の無い$t$検定（等分散）
やや古い統計学の入門書では，「データの等分散性を検定した後，分散が異なることを仮定した$t$検定か，分散が等しいことを仮定した$t$検定を使い分ける」と書かれていることがあります．  
しかし，わざわざ等分散かどうかを調べるまでもなく，常に分散が異なることを仮定した$t$検定を使っても支障ありません．すなわち，常にWelchの検定を使ってよいということです．

`stats.ttest_ind`関数に『`equal_var=False`』と指定しました．これは分散が異なることを仮定した$t$検定を行うという指定です．この指定をすると，Welchの方法が採用されます．

### $p$ハッキング
平均値の差の検定という，1つの目的のためにも，複数の検定手法がありました．対応のある$t$検定と対応の無い$t$検定では，$p$値が異なります．  
実はというと，平均値の差の検定を行う手法はほかにもいくつか知られています（マインホイットニーのU検定など）．もちろん，有意差の出やすさは各々の検定手法により異なります．

有意差が出ると嬉しいと思うタイミングはしばしばあります．  
例えば，魚の体長を大きくするために新しい餌を開発していることにします．開発された餌を使った場合と，普通の餌を使った場合とで魚の体長を比較します．ここで有意差が出れば，商品間発は大成功です．  
しかし，不等分散を仮定した$t$検定では，$p$値が0.053となってしまい，ぎりぎり有意差が得られなかったとします．  
この時に，有意差が得られるまで，何度も何度も検定の手法を切り替えていく人がいます．例えば次はU検定を使おうとか，データを変換してから検定しようとか，データを一部取り除いてから検定しようとか，有意差が得られるまで調査を続けてデータを増やそうとか．  
そうやって$p$値が0.049となり有意差が得られたとしましょう．その結果を論文に載せたとしても「検定手法をさまざま変えて，有意差が出る手法を選んだ」ことがばれてしまうことはあまりありません．  
しかし，このようなやり方で得られた有意差に”意味がある”といえるでしょうか．  
有意差は，サンプルサイズやデータのばらつきなどさまざまな特徴を包括的に取りまとめて”意味が有るかどうか”を判断する考え方だったはずです．  
それを$p$値という数値だけにこだわって「自分の欲しい結果を得るための分析」にしてしまうことは避けなければなりません．

$p$値を恣意的に変化させることを$p$ハッキングと呼びます．例えば粕谷(1998)では，いくつかの$p$ハッキングの手法が挙げられています．データの捏造や改ざんを行うまでもなく，分析の手法をこねくり回すだけで，存外簡単に$p$値を変化させることができます．  
$p$ハッキングが行われている個別の案件を告発することは困難です．予防するための有効な枠組みも見当たりません．そもそもの統計的仮説検定という手法を禁止にすべきだという意見さえあります．

データの分析は「欲しい結果」を得るために行うものではありません．  
データの分析は「現実」を知るためのものです．  
他の人がみんなやっているとか，他の人が$p$値に文句をつけてきたとか，そういうことがあっても，$p$ハッキングをしない勇気が必要です．  
**分析者は，他の対象にはどうであれ，少なくともただ１つ，データにだけは誠実でいてほしいと願います．**

・粕谷英一(1998)「生物学を学ぶ人のための統計のはなし～君にも出せる有意差～」，文一総合出版